In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.utils import shuffle
import sklearn
import glob
import cv2
import pdb
import cv2

In [2]:
#Load data function used combination data from GTI and KITTI
def loadData():
    images=glob.glob('./vehicles/vehicles/GTI_Far/image*.png')
    train_car=[]    
    train_not_car=[]
#     all data was read using cv2.imread() to avoid diffrent imge format

    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)    
        train_car.append(img)
        
    images=glob.glob('./vehicles/vehicles/GTI_Left/image*.png')
    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)  
        train_car.append(img)
        
    images=glob.glob('./vehicles/vehicles/GTI_Right/image*.png')

    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        train_car.append(img)
        

    images=glob.glob('./vehicles/vehicles/MiddleClose/image*.png')
    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        train_car.append(img)
        

    images=glob.glob('./vehicles/vehicles/KITTI_extracted/*.png')
    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        train_car.append(img)
        


    images=glob.glob('./non-vehicles/non-vehicles/GTI/image*.png')
    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        train_not_car.append(img)
        

    images=glob.glob('./non-vehicles/non-vehicles/Extras/extra*.png')
    for fname in images:
        img=cv2.imread(fname)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        train_not_car.append(img)
      

   
    return (train_car,train_not_car)


In [3]:
from mpl_toolkits.mplot3d import Axes3D
# this function to draw color space in 3D
def plot3d(pixels, colors_rgb,
        axis_labels=list("RGB"), axis_limits=[(0, 255), (0, 255), (0, 255)]):
    """Plot pixels in 3D."""

    # Create figure and 3D axes
    fig = plt.figure(figsize=(8, 8))
    ax = Axes3D(fig)

    # Set axis limits
    ax.set_xlim(*axis_limits[0])
    ax.set_ylim(*axis_limits[1])
    ax.set_zlim(*axis_limits[2])

    # Set axis labels and sizes
    ax.tick_params(axis='both', which='major', labelsize=14, pad=8)
    ax.set_xlabel(axis_labels[0], fontsize=16, labelpad=16)
    ax.set_ylabel(axis_labels[1], fontsize=16, labelpad=16)
    ax.set_zlabel(axis_labels[2], fontsize=16, labelpad=16)

    # Plot pixel values with colors given in colors_rgb
    ax.scatter(
        pixels[:, :, 0].ravel(),
        pixels[:, :, 1].ravel(),
        pixels[:, :, 2].ravel(),
        c=colors_rgb.reshape((-1, 3)), edgecolors='none')

    return ax  # return Axes3D object for further manipulatio

In [4]:
def bin_spatial(img, size=(16,16)):
    # binning spatial feature with size (16X16) to reduce computation
    features=[]    
    features .append(cv2.resize(img,size).ravel()) 
    return features
    # Return the feature vector
   

In [5]:
import numpy as np
def HistofClolor(img,nbins=32, bins_range=(0, 256)):
#     Color feature histogram
    hist_features=[]
    hhist=np.histogram(img[:,:,0], bins=32, range=(0, 255))
    shist=np.histogram(img[:,:,1], bins=32, range=(0, 255))
    vhist=np.histogram(img[:,:,2], bins=32, range=(0, 255))
    hist_features.append(np.concatenate((hhist[0], shist[0], vhist[0]))) #concatinate color features    
    return hist_features   
    

In [6]:
def get_hog_features(imgo, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False):
    
#     extracting Hog feature from gray scal image (for single channel) to reduce computation
# the commetted lines for extracting Hog for 3 three channels
    features_data = []
    if vis == True:
        img=cv2.cvtColor(imgo,cv2.COLOR_RGB2GRAY)  #conver to gray
#         ch_0=img[:,:,0]
#         ch_1=img[:,:,1]
#         ch_2=img[:,:,2]
        
#         plt.imshow(img)
#         plt.show()

        # Use skimage.hog() to get both features and a visualization
        features_0, imagehog = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell)
                                  , cells_per_block=(cell_per_block, cell_per_block),
                                  visualise=True, feature_vector=False,transform_sqrt=True )
#         plt.title('sample HOG')
#         plt.imshow(imagehog)
#         plt.savefig('./output_images/hog_sample.png')
#         plt.show()


#         features_1, imagehog = hog(ch_1, orientations=orient,
#                                   pixels_per_cell=(pix_per_cell, pix_per_cell)
#                                   , cells_per_block=(cell_per_block, cell_per_block),
#                                   visualise=True, feature_vector=False)
# #         plt.imshow(imagehog)
# #         plt.show()
# #         

#         features_2, imagehog = hog(ch_2, orientations=orient,
#                                   pixels_per_cell=(pix_per_cell, pix_per_cell)
#                                   , cells_per_block=(cell_per_block, cell_per_block),
#                                   visualise=True, feature_vector=False)
#         plt.imshow(imagehog)
#         plt.show()
        
        features_0=features_0.reshape(-1)  # reshape feature vector
#         features_1=features_1.reshape(-1)
#         features_2=features_2.reshape(-1)
        

#         features_data.append(np.concatenate((features_0,features_1,features_2)))
        features_data.append(features_0)  # appending Hog features

        
        return features_data


    else:      
        # Use skimage.hog() to get features only
        features = [] # Remove this line
        return features_data

In [7]:
# all features extraction

def extractFeature(data, color_space='HSV', size=(32, 32)):

#     Hog parameter
    orient = 9
    pix_per_cell = 8
    cell_per_block = 4
    
    
    file_features=[]  # concatinated features per image
  
    features=[]   # all features data
    feature_hog=[]  #Hog features
    spatial_size=(16,16)  #spatial binned size
#     HSV_data=[]
    count=0
    for i in data:
       
        
        img_YCrCb = cv2.cvtColor(i, cv2.COLOR_RGB2YCrCb)  #convert image to YCrCb
 #         pdb.set_trace()
        spatial_features=bin_spatial(img_YCrCb, size=spatial_size)  #extract spatial features
#         pdb.set_trace()
        hist_features=HistofClolor(img_YCrCb,nbins=32, bins_range=(0, 256))  #extract histogram features
        feature_hog=get_hog_features(i, orient,                             #extract Hog features
                        pix_per_cell, cell_per_block, 
                        vis=True, feature_vec=False)
        feature_hog=np.reshape(feature_hog,(1,-1))               #reshape Hog features
        file_features=np.concatenate((spatial_features, 
                                      hist_features,feature_hog),1)  #concatenate features image
        features.append(np.concatenate(file_features))  #append features image
    
    
    return features
    


In [8]:
import pdb

car,not_cor=loadData()  # load data
sc=car[0:4000]          #select 4000 car sample
snc=not_cor[0:4000]     #select 4000 not car sample
# plt.title('sample car')
# plt.imshow(sc[0])
# plt.savefig('./output_images/sample_car.png')
# plt.show()
# plt.title('sample not car')
# plt.imshow(snc[0])
# plt.savefig('./output_images/sample_not_car.png')
# plt.show()
#pdb.set_trace()


In [9]:
car_features=extractFeature(                     #extract car features
    sc, color_space='YCrCb', size=(32, 32))
#pdb.set_trace()
not_car_features=extractFeature(                 #extract not car features
    snc, color_space='YCrCb', size=(32, 32))

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [10]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn import svm, grid_search, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import time

# Create an array stack, NOTE: StandardScaler() expects np.float64

X = np.vstack((car_features, not_car_features)).astype(np.float64)  # stack features 

from sklearn.preprocessing import  StandardScaler
# Fit a per-column scaler
X_scaler = StandardScaler(copy = False).fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)    #Normalise data
spatial=32
histbin=32
y = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))   #create lable array
rand_state = np.random.randint(0, 1000)
X_train, X_test, y_train, y_test = train_test_split(                           #split data to train and test 40%
    scaled_X, y, test_size=0.4, random_state=rand_state)
X_train, y_train = shuffle(X_train, y_train)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))

parameters = {'kernel':['linear'], 'C':[100]}
# svc = svm.SVC()
# gs = GridSearchCV(svc, parameters,cv=2,n_jobs=4,verbose=3,)         #GridsearchCV was very slow
# gs.fit(X_train, y_train)

# aGrid = aML_GS.GridSearchCV( aClassifierOBJECT, param_grid = aGrid_of_parameters,
#                             cv = cv, n_jobs = n_JobsOnMultiCpuCores, verbose = 5 )


# # Use a linear SVC 
svc = LinearSVC()     #create classifier
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)     #fit train data
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))  #predict test data
# Check the prediction time for a single sample
t=time.time()
n_predict = len(X_test)
# print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
# print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Using spatial binning of: 32 and 32 histogram bins
Feature vector length: 4464
2.02 Seconds to train SVC...
Test Accuracy of SVC =  0.9788
9e-05 Seconds to predict 3200 labels with SVC


In [11]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    draw_img = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return draw_img

In [12]:
def windowex(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
#     pdb.set_trace()
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    #print(window_list[0].shape)
    #finalimage=draw_boxes(img,window_list)
    # Return the list of windows
#     print (len(window_list))
    return window_list


In [13]:
from sklearn.preprocessing import  StandardScaler
def searchwindow(img,ws):    #predict windows

    on_windows=[]
 
    li=[]
    for window in ws:
        test_img = cv2.resize(img[window[0][1]:window[1][1], 
                                  window[0][0]:window[1][0]], (64, 64))   #extract window     
        li.append(test_img)        #I send window in list as feature extraction funtion accept list of images
        features=extractFeature(li, color_space='RGB', size=(32, 32))    #extract features window
#         print(features[0].shape)

        test_features = X_scaler.transform(np.array(features).reshape(1, -1))  #normalise features
        prediction = svc.predict(test_features)                                #predict window
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)          #if car append in on windows
        del li[:]

    return on_windows
    
    

In [14]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

In [15]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

In [16]:

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img



In [17]:
def region_of_interestT(img):
    
    
    mask = np.zeros(img.shape, dtype=np.uint8) #mask image
    roi_corners = np.array([[(650,680), (1280,680), (1280,400),(650,400)]], 
                           dtype=np.int32) # vertisies seted to form trapezoidal scene
    channel_count = 1#img.shape[2]  # image channels
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)   
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

In [ ]:
def findcont(img):   #find contours in heat map and dilate them to group closed high heats
    imagebg=cv2.imread('./test_images/test2.jpg')
#     plt.imshow(imagebg)
#     plt.show()
    graybg=cv2.cvtColor(imagebg,cv2.COLOR_BGR2GRAY)
    grayf=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    roibg=region_of_interestT(graybg)
    roif=region_of_interestT(grayf)
    
    
    diff=cv2.subtract(roif,roibg)
    diff=np.asarray(diff,dtype=np.uint8)

    ret,thresh1 = cv2.threshold(im,20,255,cv2.THRESH_BINARY)
#     plt.imshow(diff,cmap= 'gray')
#     plt.show()
    windows=[]       
   
    im2, cnts, hierarchy = cv2.findContours(diff,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #find contours
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    cari = None
    
    for c in cnts:              #obtain four point window to be drawn       
        (x,y,w,h) = cv2.boundingRect(c)
#         cv2.rectangle(img, (x-20,y-20), (x+w,y+h), (255, 0, 0), 2)
        w=((x,y),(x+w,y+h))
        windows.append(w)

    return windows    

In [21]:
from moviepy.editor import *
from moviepy.editor import VideoFileClip
from moviepy.video.VideoClip import VideoClip
import moviepy
import matplotlib as mpimg
import time
from moviepy.Clip import Clip
from scipy.ndimage.measurements import label
from Lane import *
import scipy.misc



clip = VideoFileClip('project_video.mp4') # Read clip
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')      #OpenCv writing file
out = cv2.VideoWriter('out_video_c.avi',fourcc, 15.0, (640, 360)) # intialise output file

frameClibration= clip.get_frame(0)

im=scipy.misc.imresize(frameClibration, (180, 320,3))


#im=scipy.misc.imresize(rdist, (180, 320,3))
font = cv2.FONT_HERSHEY_SIMPLEX

d=0      # Frame number
# hot_windows=[]
for frame in clip.iter_frames(dtype=np.uint8):
    
        
    imageX=frame

    heat = np.zeros_like(frame[:,:,0]).astype(np.float) # create heat map
    t=time.time()

#extract multiscale windows
    w=findcont(frame)
    

    hot_windows=searchwindow(imageX,w)

    

 ##############################################
    heat = add_heat(heat,hot_windows)

#         Apply threshold to help remove false positives
    heat = apply_threshold(heat,4)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

#         w=findcont(heatmap)     # other method for group closed hot peaks  
    labels = label(heatmap)
    draw_img2=draw_boxes(np.copy(frame),hot_windows)

#     fig = plt.figure()
#     plt.subplot(121)
#     plt.imshow(draw_img2)
#     plt.title('Car Positions2')
#     plt.subplot(122)
#     plt.imshow(heatmap, cmap='hot')
#     plt.title('Heat Map')
#     fig.tight_layout()
#     plt.savefig('./output_images/Detected_cars.png')
#     plt.show()



   
    t2 = time.time()
    tf=t2-t
    draw_img2=cv2.putText(draw_img2,'time per frame: %.1f m'%tf,(50,200),
                   font, 1,(255,255,255),2,cv2.LINE_AA)

#     plt.imshow(final_image)
# #     plt.savefig('./output_images/result.png')
#     plt.show()

    f=cv2.resize(draw_img2,(640,360))    #resize frame to reduce video file size 

    out.write(f)


    print('Process time per frame',d,':..',round(t2-t, 5))
#     print (d)
    d=d+1
out.release()
    

Process time per frame 0 :.. 0.21504
Process time per frame 1 :.. 0.23592
Process time per frame 2 :.. 0.23498
Process time per frame 3 :.. 0.23476
Process time per frame 4 :.. 0.23508
Process time per frame 5 :.. 0.23457
Process time per frame 6 :.. 0.23371
Process time per frame 7 :.. 0.23388
Process time per frame 8 :.. 0.23484
Process time per frame 9 :.. 0.23424
Process time per frame 10 :.. 0.23406
Process time per frame 11 :.. 0.23447
Process time per frame 12 :.. 0.23222
Process time per frame 13 :.. 0.23423
Process time per frame 14 :.. 0.23267
Process time per frame 15 :.. 0.23346
Process time per frame 16 :.. 0.23325
Process time per frame 17 :.. 0.23254
Process time per frame 18 :.. 0.23287
Process time per frame 19 :.. 0.23349
Process time per frame 20 :.. 0.233
Process time per frame 21 :.. 0.23203
Process time per frame 22 :.. 0.23358
Process time per frame 23 :.. 0.2339
Process time per frame 24 :.. 0.2343
Process time per frame 25 :.. 0.23447
Process time per frame 26 

Process time per frame 214 :.. 0.22933
Process time per frame 215 :.. 0.22854
Process time per frame 216 :.. 0.22986
Process time per frame 217 :.. 0.2293
Process time per frame 218 :.. 0.22904
Process time per frame 219 :.. 0.22849
Process time per frame 220 :.. 0.2289
Process time per frame 221 :.. 0.2272
Process time per frame 222 :.. 0.22875
Process time per frame 223 :.. 0.22839
Process time per frame 224 :.. 0.22809
Process time per frame 225 :.. 0.2283
Process time per frame 226 :.. 0.22908
Process time per frame 227 :.. 0.22904
Process time per frame 228 :.. 0.22873
Process time per frame 229 :.. 0.23011
Process time per frame 230 :.. 0.22905
Process time per frame 231 :.. 0.22955
Process time per frame 232 :.. 0.22873
Process time per frame 233 :.. 0.2288
Process time per frame 234 :.. 0.22891
Process time per frame 235 :.. 0.22898
Process time per frame 236 :.. 0.22764
Process time per frame 237 :.. 0.22735
Process time per frame 238 :.. 0.22898
Process time per frame 239 :..

Process time per frame 425 :.. 0.23165
Process time per frame 426 :.. 0.23162
Process time per frame 427 :.. 0.23204
Process time per frame 428 :.. 0.23178
Process time per frame 429 :.. 0.23034
Process time per frame 430 :.. 0.23148
Process time per frame 431 :.. 0.2334
Process time per frame 432 :.. 0.23333
Process time per frame 433 :.. 0.23364
Process time per frame 434 :.. 0.23346
Process time per frame 435 :.. 0.23263
Process time per frame 436 :.. 0.23256
Process time per frame 437 :.. 0.23222
Process time per frame 438 :.. 0.23128
Process time per frame 439 :.. 0.23061
Process time per frame 440 :.. 0.23085
Process time per frame 441 :.. 0.23058
Process time per frame 442 :.. 0.23092
Process time per frame 443 :.. 0.23318
Process time per frame 444 :.. 0.23157
Process time per frame 445 :.. 0.2339
Process time per frame 446 :.. 0.23273
Process time per frame 447 :.. 0.23222
Process time per frame 448 :.. 0.23173
Process time per frame 449 :.. 0.23204
Process time per frame 450 

Process time per frame 636 :.. 0.22792
Process time per frame 637 :.. 0.22821
Process time per frame 638 :.. 0.22897
Process time per frame 639 :.. 0.22913
Process time per frame 640 :.. 0.23029
Process time per frame 641 :.. 0.22974
Process time per frame 642 :.. 0.23131
Process time per frame 643 :.. 0.23131
Process time per frame 644 :.. 0.23129
Process time per frame 645 :.. 0.23166
Process time per frame 646 :.. 0.23231
Process time per frame 647 :.. 0.23211
Process time per frame 648 :.. 0.23185
Process time per frame 649 :.. 0.23283
Process time per frame 650 :.. 0.23301
Process time per frame 651 :.. 0.2308
Process time per frame 652 :.. 0.23321
Process time per frame 653 :.. 0.23363
Process time per frame 654 :.. 0.2303
Process time per frame 655 :.. 0.23269
Process time per frame 656 :.. 0.23214
Process time per frame 657 :.. 0.2322
Process time per frame 658 :.. 0.23299
Process time per frame 659 :.. 0.23295
Process time per frame 660 :.. 0.23329
Process time per frame 661 :

Process time per frame 847 :.. 0.23306
Process time per frame 848 :.. 0.23368
Process time per frame 849 :.. 0.23281
Process time per frame 850 :.. 0.23481
Process time per frame 851 :.. 0.23357
Process time per frame 852 :.. 0.2332
Process time per frame 853 :.. 0.23294
Process time per frame 854 :.. 0.23086
Process time per frame 855 :.. 0.23233
Process time per frame 856 :.. 0.2328
Process time per frame 857 :.. 0.23088
Process time per frame 858 :.. 0.23296
Process time per frame 859 :.. 0.23448
Process time per frame 860 :.. 0.23206
Process time per frame 861 :.. 0.2341
Process time per frame 862 :.. 0.23472
Process time per frame 863 :.. 0.23375
Process time per frame 864 :.. 0.23375
Process time per frame 865 :.. 0.23117
Process time per frame 866 :.. 0.23329
Process time per frame 867 :.. 0.23117
Process time per frame 868 :.. 0.23368
Process time per frame 869 :.. 0.23236
Process time per frame 870 :.. 0.23467
Process time per frame 871 :.. 0.2334
Process time per frame 872 :.

Process time per frame 1057 :.. 0.22944
Process time per frame 1058 :.. 0.22936
Process time per frame 1059 :.. 0.22913
Process time per frame 1060 :.. 0.22905
Process time per frame 1061 :.. 0.22876
Process time per frame 1062 :.. 0.22762
Process time per frame 1063 :.. 0.23026
Process time per frame 1064 :.. 0.22875
Process time per frame 1065 :.. 0.22934
Process time per frame 1066 :.. 0.23082
Process time per frame 1067 :.. 0.23009
Process time per frame 1068 :.. 0.23089
Process time per frame 1069 :.. 0.23089
Process time per frame 1070 :.. 0.2307
Process time per frame 1071 :.. 0.23063
Process time per frame 1072 :.. 0.22948
Process time per frame 1073 :.. 0.23042
Process time per frame 1074 :.. 0.2298
Process time per frame 1075 :.. 0.22956
Process time per frame 1076 :.. 0.22957
Process time per frame 1077 :.. 0.22973
Process time per frame 1078 :.. 0.22969
Process time per frame 1079 :.. 0.23084
Process time per frame 1080 :.. 0.23035
Process time per frame 1081 :.. 0.23001
Pr